In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

TOP_POSTER_LIMIT = 10
DOMAIN_LIMIT = 25
REPLIES_THRESHOLD = 150

df = pd.read_csv("data/latest-articles.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
df["title"] = df["title"].astype("category")
df["posted_by"] = df["posted_by"].astype("category")
df["source_url"] = df["source_url"].astype("category")

In [2]:
# A thread is considered sourced if it has a URL link.
sourced = df[df.source_url != ""]

# This is not entirely accurate since some vanity posts have links.
vanity = df[df.source_url == ""]

In [3]:
def extract_domain(url) -> str:
    return (
        url.
            replace("www.","").
            replace("http://", "").
            replace("https://", "").
            split("/")[0]
    )


sourced.insert(len(sourced.columns)-1, "domain", (
    sourced.
        source_url.
        apply(extract_domain).
        astype("category")
))

In [4]:
# Draws a chart that shows where discussion for a particular thread originated from.
data = sourced[["source_url", "domain"]].drop_duplicates()
top_sourced_domains = data.domain.value_counts().iloc[:DOMAIN_LIMIT].index
sns.countplot(
    y="domain",
    data=data,
    order=top_sourced_domains,
)

<Axes: xlabel='count', ylabel='domain'>

In [5]:
# Draws a chart that shows who posted a topic to be discussed that has a URL source.
data = sourced[["source_url", "posted_by"]].drop_duplicates()
top_sourced_posters = data.posted_by.value_counts().iloc[:TOP_POSTER_LIMIT].index
sns.countplot(
    y="posted_by",
    data=data,
    order=top_sourced_posters,
)

<Axes: xlabel='count', ylabel='posted_by'>

In [6]:
# Comment this if you don't want to see everything.
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Draws a table that shows which freepers posted topics from a sourced link that originated
# from a particular domain.
who_is_posting_from_what = (
    sourced[["posted_by", "source_url", "domain"]].
    query("posted_by in @top_sourced_posters and domain in @top_sourced_domains").
    drop_duplicates().
    groupby(["domain", "posted_by"])
)
who_is_posting_from_what[["domain"]].describe()

/tmp/ipykernel_22799/2178239382.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby(["domain", "posted_by"])


domain         \
                                                count unique   
domain                      posted_by                          
americanthinker.com         E. Pluribus Unum       40      1   
                            MtnClimber             88      1   
                            SeekAndFind            21      1   
                            george76                2      1   
apnews.com                  E. Pluribus Unum       57      1   
                            SeekAndFind             2      1   
                            george76                1      1   
                            DFG                     2      1   
                            MinorityRepublican      4      1   
                            marcusmaximus           1      1   
babylonbee.com              george76                1      1   
                            DFG                    15      1   
                            marcusmaximus           1      1   
bbc.com                     E. Pluribus Unum        3      1   
                            MinorityRepublican     14      1   
                            marcusmaximus           1      1   
                            Morgana                 1      1   
                            Red Badger              1      1   
                            nickcarraway            7      1   
breitbart.com               DFG                     6      1   
                            MinorityRepublican      2      1   
                            Morgana                 5      1   
                            Red Badger             15      1   
dailymail.co.uk             E. Pluribus Unum        6      1   
                            DFG                    19      1   
                            MinorityRepublican     21      1   
                            marcusmaximus           4      1   
                            Morgana                48      1   
                            Red Badger              2      1   
                            nickcarraway            3      1   
foxnews.com                 E. Pluribus Unum        4      1   
                            MtnClimber              2      1   
                            george76                4      1   
                            MinorityRepublican     16      1   
                            marcusmaximus           1      1   
                            Morgana                 1      1   
                            Red Badger              1      1   
                            nickcarraway            2      1   
justthenews.com             E. Pluribus Unum       26      1   
                            MtnClimber              2      1   
                            george76                6      1   
                            Red Badger              4      1   
latimes.com                 E. Pluribus Unum       32      1   
                            george76                1      1   
                            MinorityRepublican      2      1   
msn.com                     E. Pluribus Unum        1      1   
                            SeekAndFind             5      1   
                            marcusmaximus          15      1   
                            Morgana                 1      1   
                            Red Badger              1      1   
nationalsecurityjournal.org Red Badger              2      1   
notthebee.com               DFG                     2      1   
                            Red Badger             42      1   
nypost.com                  E. Pluribus Unum        2      1   
                            MtnClimber              2      1   
                            george76               11      1   
                            DFG                    25      1   
                            MinorityRepublican     14      1   
                            marcusmaximus          17      1   
                            Morgana                 2      1   
               

In [7]:
# Display which posts got the most replies before
# they aged out from the front page.

(sourced[['timestamp', 'title', 'replies']].
     loc[sourced.replies > REPLIES_THRESHOLD].
     groupby('title').
     max().
     sort_values('replies', ascending=False).
     dropna())

/tmp/ipykernel_22799/1921096508.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby('title').


timestamp  \
title                                                                    
Trump posts on truth, social in defensive Pam B... 2025-07-13 02:43:53   
TRUMP ISSUES MESSAGE OF TOTAL SUPPORT FOR PAM B... 2025-07-13 00:38:41   
Favorite Boy Or Girl Name Songs.Tell us your Fa... 2025-07-11 03:34:25   
“They’re Coming Out of the Woodwork!” Whistlebl... 2025-07-21 00:03:11   
Did the DOGE layoffs at the National Weather Se... 2025-07-06 03:38:06   
'It's a cover up': Musk floods X with posts att... 2025-07-18 01:20:38   
EXCLUSIVE: Lara Trump says the Trump administra... 2025-07-15 00:47:08   
Trump: ‘We have to’ send Ukraine more weapons      2025-07-08 04:06:02   

                                                    replies  
title                                                        
Trump posts on truth, social in defensive Pam B...    228.0  
TRUMP ISSUES MESSAGE OF TOTAL SUPPORT FOR PAM B...    214.0  
Favorite Boy Or Girl Name Songs.Tell us your Fa...    190.0  
“They’re Coming Out of the Woodwork!” Whistlebl...    172.0  
Did the DOGE layoffs at the National Weather Se...    166.0  
'It's a cover up': Musk floods X with posts att...    163.0  
EXCLUSIVE: Lara Trump says the Trump administra...    160.0  
Trump: ‘We have to’ send Ukraine more weapons         154.0